<a href="https://colab.research.google.com/github/ramincre/Dual-Path-Transformer-Network-PyTorch/blob/main/BSS_DPTNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get update -y

!sudo apt-get install python3.8

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

!sudo update-alternatives --config python3

!apt-get install python3-pip

!python -m pip install --upgrade pip --user

!sudo apt-get install --reinstall python3.8-distutils

In [ ]:
!python --version

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/Dataset/MiniLibriMix.rar" "/content"

In [ ]:
!unrar x /content/MiniLibriMix.rar

In [4]:
import os

In [ ]:
!pip install librosa
!pip install torch

In [5]:
import torch
torch.cuda.is_available()

True

In [ ]:
#!wget https://zenodo.org/records/3871592/files/MiniLibriMix.zip?download=1

In [ ]:
#os.rename('/content/MiniLibriMix.zip?download=1','/content/MiniLibriMix.zip')

In [6]:
!git clone https://github.com/ramincre/Dual-Path-Transformer-Network-PyTorch.git

Cloning into 'Dual-Path-Transformer-Network-PyTorch'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 292 (delta 63), reused 130 (delta 63), pack-reused 162
Receiving objects: 100% (292/292), 2.61 MiB | 5.91 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [ ]:
"""
os.rename('/content/MiniLibriMix','/content/WSJ2mix')
os.rename('/content/WSJ2mix/train','/content/WSJ2mix/tr')
os.rename('/content/WSJ2mix/val','/content/WSJ2mix/cv')
os.rename('/content/WSJ2mix/tr/mix_both','/content/WSJ2mix/tr/mix')
os.rename('/content/WSJ2mix/cv/mix_both','/content/WSJ2mix/cv/mix')
"""

In [7]:
!mkdir /content/output
"""
!mkdir /content/WSJ2mix/tt
!mkdir /content/WSJ2mix/tt/mix
!mkdir /content/WSJ2mix/tt/s1
!mkdir /content/WSJ2mix/tt/s2
"""

'\n!mkdir /content/WSJ2mix/tt\n!mkdir /content/WSJ2mix/tt/mix\n!mkdir /content/WSJ2mix/tt/s1\n!mkdir /content/WSJ2mix/tt/s2\n'

In [8]:
!python /content/Dual-Path-Transformer-Network-PyTorch/preprocess.py --in-dir /content/MiniLibriMix --out-dir /content/output --sample-rate 8000

Namespace(in_dir='/content/MiniLibriMix', out_dir='/content/output', sample_rate=8000)


In [13]:
!python /content/Dual-Path-Transformer-Network-PyTorch/train.py \
--train_dir '/content/output/tr'\
 --valid_dir '/content/output/cv'\
 --epochs 20 \
 --batch_size 3 \
 --num_workers 2  \
 --N 16

Namespace(train_dir='/content/output/tr', valid_dir='/content/output/cv', sample_rate=8000, segment=4, cv_maxlen=8, N=16, C=2, L=4, H=4, K=250, B=6, use_cuda=1, epochs=20, half_lr=0, early_stop=0, max_norm=5, shuffle=0, batch_size=3, num_workers=2, optimizer='adam', lr=0.125, momentum=0.0, l2=0.0, save_folder='exp/temp', checkpoint=0, continue_from='', model_path='final.pth.tar', print_freq=1000, visdom=0, visdom_epoch=0, visdom_id='TasNet training')
Drop 38 utts(0.00 h) which is short than 32000 samples
# of parameters: 180609
Training...
Epoch 1 | Iter 1 | Average Loss 7.428 | Current Loss 7.428497 | 7286.8 ms/batch
-------------------------------------------------------------------------------------
Train Summary | End of Epoch 1 | Time 161.52s | Train Loss 8.275
-------------------------------------------------------------------------------------
Cross validation...
Epoch 1 | Iter 1 | Average Loss 3.033 | Current Loss 3.033265 | 2126.7 ms/batch
-------------------------------------

In [ ]:
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [20]:
!pip install mir_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100703 sha256=7becffd03d391c3dd545b4d229ab1ada10b8f7e00cd8fa990676830bfa352ca6
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built mir_eval


In [22]:
!python /content/Dual-Path-Transformer-Network-PyTorch/evaluate.py \
--data_dir '/content/output/tt' \
--model_path '/content/exp/temp/temp_best.pth.tar'\
--N 16 \
--batch_size 3

Namespace(N=16, C=2, L=4, H=4, K=250, B=6, model_path='/content/exp/temp/temp_best.pth.tar', data_dir='/content/output/tt', cal_sdr=1, use_cuda=0, sample_rate=8000, batch_size=3)
DPTNet(
  (encoder): Encoder(
    (conv1d_U): Conv1d(1, 16, kernel_size=(4,), stride=(2,), bias=False)
  )
  (separator): Separator(
    (LN): GroupNorm(1, 16, eps=1e-08, affine=True)
    (DPT): ModuleList(
      (0-5): 6 x DPTBlock(
        (intra_transformer): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): LSTM(16, 32, bidirectional=True)
          (linear2): Linear(in_features=64, out_features=16, bias=True)
          (dropout): Dropout(p=0, inplace=False)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0, inplace=False)
         

In [ ]:
!python /content/Dual-Path-Transformer-Network-PyTorch/separate.py\
 --mix_json '/content/output/tt/mix.json' \
 --model_path '/content/exp/temp/temp_best.pth.tar'\
 --N 16 \
--batch_size 3

Namespace(N=16, C=2, L=4, H=4, K=250, B=6, model_path='/content/exp/temp/temp_best.pth.tar', mix_dir=None, mix_json='/content/output/tt/mix.json', out_dir='exp/result', use_cuda=0, sample_rate=8000, batch_size=3)
